In [ ]:
%matplotlib inline
from pprint import pprint
from decimal import Decimal
import itertools
import metrics
import pandas

import simulate
import harvesting
import market
import withdrawal
import numpy

In [ ]:
import seaborn
from matplotlib import pyplot as plt
import matplotlib
seaborn.set(style="whitegrid")
seaborn.set_context('poster')

In [ ]:
series = market.Returns_US_1871()

In [ ]:
TYPE = 'Withdrawal (Real)'
m = {'Withdrawal (Nominal)': lambda x: float(x.withdraw_n),
     'Withdrawal (Real)' : lambda x: float(x.withdraw_r),
     'Portfolio (Nominal)': lambda x: float(x.portfolio_n),
     'Portfolio (Real)': lambda x: float(x.portfolio_r),
}
lens = m[TYPE]
lens.__label__ = TYPE

def fn(x):
    return [lens(_) for _ in x]

In [ ]:
VPW_YEARS=40

def run_sim(year, key, length=30):
    w = {
        'raw' : lambda p, s: withdrawal.VPW(p, s, years_left=VPW_YEARS),
        'cape10' : lambda p, s: withdrawal.CAPE10Smoothing(year, withdrawal.VPW(p, s, years_left=VPW_YEARS)),
    }

    portfolio = (500000,500000)
    harvest_strategy = harvesting.make_rebalancer(0.5)
    
    return simulate.withdrawals(series.iter_from(year),
                                 withdraw=w[key],
                                 years=length,
                                 portfolio=portfolio,
                                 harvesting=harvest_strategy)

In [ ]:
def chart_one(year):
    pmt = run_sim(year, 'raw')
    cape10 = run_sim(year, 'cape10')
    
    df = pandas.DataFrame({
        'pmt' : [lens(x) for x in pmt],
        'cape' : [lens(x) for x in cape10],
    })
    g = seaborn.relplot(
            data=df,
            kind='line',
            aspect=2,
        )
    g.fig.autofmt_xdate()
    g.despine(left=True, bottom=True, offset=20)
    g.fig.suptitle(f'Retirement in {year}')
    return g

In [ ]:
chart_one(1980)

In [ ]:
def run_all():
    length = 30
    
    all = pandas.DataFrame(index=range(1881, 2018-length+1), columns=['pmt', 'cape'])
    
    for i in range(1881, 2018-length+1):
        pmt = run_sim(i, 'raw', length=length)
        cape10 = run_sim(i, 'cape10', length=length)
        
        df = pandas.DataFrame({
            'pmt' : [lens(x) for x in pmt],
            'cape' : [lens(x) for x in cape10]
        })
        
        all.loc[i]['pmt'] = df['pmt'].std()
        all.loc[i]['cape'] = df['cape'].std()

        return all

In [ ]:
df = run_all()
df.head()

In [ ]:
g = seaborn.relplot(
        data=df,
#        kind='line',
        aspect=2,
    )
g.fig.autofmt_xdate()
g.despine(left=True, bottom=True, offset=20)
g.fig.suptitle(f'stdev of retirement income')

In [ ]:
df.mean()

In [ ]:
df.median()

In [ ]:
chart_one(1900)

In [ ]:
len(df[df['cape'] < df['pmt']])

In [ ]:
def run_all():
    length = 5
    
    all = pandas.DataFrame(index=range(1881, 2018-length+1), columns=['pmt', 'cape'])
    
    for i in range(1881, 2018-length+1):
        pmt = run_sim(i, 'raw', length=length)
        cape10 = run_sim(i, 'cape10', length=length)
        
        df = pandas.DataFrame({
            'pmt' : [lens(x) for x in pmt],
            'cape' : [lens(x) for x in cape10]
        })
        
        all.loc[i]['pmt'] = metrics.ulcer(df['pmt'][0:5])
        all.loc[i]['cape'] = metrics.ulcer(df['cape'][0:5])
        
    return all

In [ ]:
df = run_all()

In [ ]:
g = seaborn.relplot(
        data=df,
#        kind='line',
        aspect=3,
    )
g.fig.autofmt_xdate()
g.despine(left=True, bottom=True, offset=20)
g.fig.suptitle(f'Ulcer Index of Income (First 5 years)')

In [ ]:
(df['cape'] - df['pmt']).sort_values(ascending=False).head()

In [ ]:
u = df['cape'] - df['pmt']
u[u > 0].mean()

In [ ]:
def run_all():
    length = 15
    
    all = pandas.DataFrame(index=range(1881, 2018-length+1), columns=['pmt', 'cape'])
    
    for i in range(1881, 2018-length+1):
        pmt = run_sim(i, 'raw', length=length)
        cape10 = run_sim(i, 'cape10', length=length)
        
        df = pandas.DataFrame({
            'pmt' : [lens(x) for x in pmt],
            'cape' : [lens(x) for x in cape10]
        })
        
        all.loc[i]['pmt'] = Decimal(df['pmt'][0]) / metrics.cew(df['pmt'].apply(Decimal))
        all.loc[i]['cape'] = Decimal(df['cape'][0]) / metrics.cew(df['cape'].apply(Decimal))
        
    return all

In [ ]:
df = run_all()

In [ ]:
df['cape'].sort_values(ascending=False).head()

In [ ]:
df['pmt'].sort_values(ascending=False).head()

In [ ]:
chart_one(1966)

In [ ]:
chart_one(1966)

In [ ]:
df.median()

In [ ]:
df.mean()

In [ ]:
cape10_csv = pandas.read_csv('1871_returns.csv', usecols=['Year', 'CAPE10'], index_col='Year')

In [ ]:
df['CAPE10'] = cape10_csv['CAPE10']

In [ ]:
df.sort_values(by='CAPE10', ascending=False).head()

In [ ]:
df.loc[1998]

In [ ]:
chart_one(1998)